# Claycode SIGGRAPH Evaluation

In [ ]:
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv('results.csv')

# Process the 'type' column to categorize types based on the filename
df['type'] = df['filename'].str.split('-').map(lambda x: "-".join(x[:-1]).replace("_clay", "clay"))

# Select relevant columns
df = df[['successful', 'wave_amplitude', 'type']]

# Group by wave amplitude and type, then calculate success rate
grouped = df.groupby(['wave_amplitude', 'type'])
df = grouped['successful'].agg(['sum', 'count']).reset_index()
df['success_rate'] = df['sum'] / df['count']
df['type'] = df['type'].replace({'claycode': 'Claycode (R=1)',
                                 'claycode-fragment': 'Claycode (R=2)',
                                 'qr': 'QR',
                                 'qrart': 'Styl. QR',
                                 'claycodeart': 'Styl. Claycode (R=1)',
                                 'barcode': 'Barcode'
                                 })

# Define the custom color scheme using Dark2[0], Dark2[1], Dark2[2]
color_scheme = {
    'Claycode (R=1)': '#1b9e77', 
    'Styl. Claycode (R=1)': '#1b9e77', 
    'Claycode (R=2)': '#1b9e77', 
    'QR': '#d95f02', 
    'Styl. QR': '#d95f02', 
    'Barcode': '#7570b3'
}

# Define line dash style
line_dash_map = {
    'Claycode (R=1)': 'solid', 
    'Styl. Claycode (R=1)': 'dot',  
    'Claycode (R=2)': 'dashdot',  
    'QR': 'solid', 
    'Styl. QR': 'dot', 
    'Barcode': 'solid'
}

# Define line width and marker symbols
line_width_map = {
    'Claycode (R=1)': 2,
    'Styl. Claycode (R=1)': 2, 
    'Claycode (R=2)': 2, 
    'QR': 2, 
    'Styl. QR': 2, 
    'Barcode': 2
}

marker_map = {
    'QR': 'square-open',
    'Styl. QR': 'square',
    'Claycode (R=1)': 'hexagram-open',
    'Styl. Claycode (R=1)': 'hexagram-dot',
    'Claycode (R=2)': 'star',
    'Barcode': 'cross',
}

# Apply small offsets to the success rate for better separation between overlapping claycode types
df['adjusted_success_rate'] = df['success_rate']
df.loc[df['type'] == 'Claycode (R=2)', 'adjusted_success_rate'] += 0.01

# Create the line plot with custom color, line styles, and width
fig = px.line(
    df,
    x="wave_amplitude",
    y="adjusted_success_rate",
    color="type",
    line_dash="type",
    markers=True,
    labels={"wave_amplitude": "Wave Amplitude (ω)", "adjusted_success_rate": "Success rate (%)", "type": "Type"},
    color_discrete_map=color_scheme,
)

# Apply line width, dash style, and marker symbols
for trace in fig.data:
    trace.line.dash = line_dash_map[trace.name]
    trace.line.width = line_width_map[trace.name]
    trace.marker.symbol = marker_map[trace.name]
    trace.marker.size = 13
    trace.marker.opacity = 0.9

# Move the 'Barcode' trace to the foreground by reordering
fig.data = sorted(fig.data, key=lambda x: x.name != 'Barcode', reverse=True)

# Update layout
fig.update_layout(
    height=400,
    width=700,
    margin=dict(t=30, b=50, l=50, r=50),
    legend=dict(
        x=0.675,
        y=0.61,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='black',
        borderwidth=1,
        title=""
    )
)

# Update axes
fig.update_xaxes(range=[-0.05, 1.05], tickformat='.')
fig.update_yaxes(range=[-0.05, 1.05], tickformat='.0%')

# Export to PDF
fig.write_image("sinusoidal-no-square.pdf", format="pdf", engine="kaleido")
fig.show()


KeyError: 'Barcode'

In [83]:
# y: every kind of string (group by message name)
# x: amplitudine
# y: type of code

df = pd.read_csv('results.csv')

# Extract the string between the last '-' and '.png' in the filename
df['extracted_value'] = df['filename'].str.extract(r'-([^-]+)\.png$')

# Calculate the length of the extracted value and use it for the Y-axis
df['extracted_value_len'] = df['extracted_value'].apply(len)

# Sort by the length of the extracted value
df = df.sort_values('extracted_value_len')

# Group by 'extracted_value_len' and 'wave_amplitude', then count the successful entries
grouped_df = df.groupby(['extracted_value_len', 'wave_amplitude'], as_index=False).agg({
    'successful': 'sum'  # Count of successful rows (True = 1, False = 0)
})

x_max = grouped_df['wave_amplitude'].max()
if x_max != 1:
    x_max = 1

# Create the heatmap with length of extracted value on Y-axis
fig = px.density_heatmap(
    grouped_df,
    x='wave_amplitude',
    y='extracted_value_len',
    z='successful',
    title='Heatmap of Wave Amplitude and Length of Extracted Filename Value (Count of Successful)',
    labels={
        'wave_amplitude': 'Wave Amplitude',
        'extracted_value_len': 'Length of Extracted Value',
        'successful': 'Count of Successful Entries'
    },
    color_continuous_scale='Teal',
)

fig.update_layout(
    xaxis=dict(range=[0, 1]),
)

# Show the plot
fig.show()